<a href="https://colab.research.google.com/github/gustavol3ma/Learning_journey/blob/main/RAG_AULA02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade langchain langchain-google-genai faiss-cpu chromadb langchain-pinecone pinecone-client langchain-community

In [ ]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('APY_RAG')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document

embeddings = GoogleGenerativeAIEmbeddings(model='gemini-embedding-001')
#Ela instancia um modelo de embeddings do Google Gemini(via LangChain),permitindo transformar
#textos em vetores numéricos de alta dimensão

In [ ]:
embeddings.embed_query("Política de home office da empresa")

[0.016149115,
 0.040602732,
 0.03302508,
 -0.05995194,
 0.0015303721,
 -0.0080236755,
 -0.0035909659,
 0.0012388312,
 0.017689722,
 -0.03018673,
 0.0025397444,
 -0.019532718,
 -0.009552416,
 0.02332649,
 0.1223549,
 0.010480531,
 0.025290024,
 0.019415487,
 -0.012007049,
 -0.0024108249,
 0.02019597,
 -0.001310815,
 0.0013476798,
 -0.02222931,
 -0.017976293,
 -0.052221924,
 0.0051751737,
 0.0058862227,
 0.027025696,
 0.005459975,
 -0.01131303,
 0.023075413,
 -0.00015346195,
 0.012506196,
 0.007758465,
 0.012851455,
 0.020559039,
 0.007812641,
 -0.00782638,
 0.018867722,
 -0.019412031,
 -0.0011542309,
 0.003912106,
 -0.011155264,
 0.019515581,
 -0.0009824866,
 -0.020222325,
 -0.0441532,
 0.018064748,
 0.0135432,
 0.021039164,
 -0.008962987,
 -0.010015963,
 -0.16120756,
 0.010549564,
 -0.0039220015,
 -0.004724151,
 0.017542338,
 -0.017530441,
 0.0007696376,
 -0.04336273,
 0.030578358,
 0.011024619,
 -0.018101402,
 -0.008776255,
 0.010636258,
 0.016787814,
 0.016862858,
 0.026166154,
 -0.0

In [ ]:
documentos_empresa = [
    Document(
        page_content="Política de férias: Funcionários têm direito a 30 dias de férias após 12 meses. A solicitação deve ser feita com 30 dias de antecedência.",
        metadata={"tipo": "política", "departamento": "RH", "ano": 2024, "id_doc": "doc001"}
    ),
    Document(
        page_content="Processo de reembolso de despesas: Envie a nota fiscal pelo portal financeiro. O reembolso ocorre em até 5 dias úteis.",
        metadata={"tipo": "processo", "departamento": "Financeiro", "ano": 2023, "id_doc": "doc002"}
    ),
    Document(
        page_content="Guia de TI: Para configurar a VPN, acesse vpn.nossaempresa.com e siga as instruções para seu sistema operacional.",
        metadata={"tipo": "tutorial", "departamento": "TI", "ano": 2024, "id_doc": "doc003"}
    ),
    Document(
        page_content="Código de Ética e Conduta: Valorizamos o respeito, a integridade e a colaboração. Casos de assédio não serão tolerados.",
        metadata={"tipo": "política", "departamento": "RH", "ano": 2022, "id_doc": "doc004"}
    )
]

In [ ]:
from langchain_community.vectorstores import FAISS
import faiss

d = 768
index_hns2= faiss.IndexHNSWFlat(d,32)

In [ ]:
faiss_db = FAISS.from_documents(documentos_empresa, embeddings)

pergunta =  "Como peço minhas férias?"
resultados = faiss_db.similarity_search_with_score(pergunta, k=2)


In [ ]:
print(f"\n Pergunta: {pergunta}")
print("\n Documento mais relevantes (FAISS):")
for doc in resultados:
  print(f"- {doc[0].page_content}")
  print(f"(Metadados: {doc[0].metadata})")


 Pergunta: Como peço minhas férias?

 Documento mais relevantes (FAISS):
- Política de férias: Funcionários têm direito a 30 dias de férias após 12 meses. A solicitação deve ser feita com 30 dias de antecedência.
(Metadados: {'tipo': 'política', 'departamento': 'RH', 'ano': 2024, 'id_doc': 'doc001'})
- Processo de reembolso de despesas: Envie a nota fiscal pelo portal financeiro. O reembolso ocorre em até 5 dias úteis.
(Metadados: {'tipo': 'processo', 'departamento': 'Financeiro', 'ano': 2023, 'id_doc': 'doc002'})


Chroma DB: Banco de Dados Vetorial Simples e Poderoso para IA

In [ ]:
from langchain_community.vectorstores import Chroma

chroma_db = Chroma.from_documents(
    documents = documentos_empresa,
    embedding = embeddings
)

resultados = chroma_db.similarity_search(pergunta, k=2)

for doc in resultados:
  print(f"- {doc.page_content}")

- Política de férias: Funcionários têm direito a 30 dias de férias após 12 meses. A solicitação deve ser feita com 30 dias de antecedência.
- Processo de reembolso de despesas: Envie a nota fiscal pelo portal financeiro. O reembolso ocorre em até 5 dias úteis.


Pergunta por Filtro


In [ ]:
pergunta_rh = "Quais são as regras da empresa?"

resultados_filtrado = chroma_db.similarity_search(pergunta_rh, k=2, filter={"$and": [{"departamento" : "RH"},{"tipo" : "política"}]})

for doc in resultados_filtrado:
  print(f"- {doc.page_content}")
  print(f"(Metadados: {doc.metadata['departamento']}, Tipo: {doc.metadata['tipo']}")

- Código de Ética e Conduta: Valorizamos o respeito, a integridade e a colaboração. Casos de assédio não serão tolerados.
(Metadados: RH, Tipo: política
- Política de férias: Funcionários têm direito a 30 dias de férias após 12 meses. A solicitação deve ser feita com 30 dias de antecedência.
(Metadados: RH, Tipo: política


Pinecone - Escalabilidade na Nuvem


 É um banco de dados vetorial de nível profissional, muito utilizado em ambientes de produção.

In [ ]:
from google.colab import userdata
userdata.get('APY_PINECONE')
os.environ['PINECONE_API_KEY'] = userdata.get('APY_PINECONE')

In [ ]:
from langchain_community.vectorstores import Pinecone
from pinecone import Pinecone as PineconeClient
from pinecone import ServerlessSpec

index_name ="langchain-rag"
pinecone_client = PineconeClient(api_key=os.environ["PINECONE_API_KEY"])
spec = ServerlessSpec(cloud='aws', region='us-east-1')

In [ ]:
!pip show langchain-pinecone

Name: langchain-pinecone
Version: 0.2.13
Summary: An integration package connecting Pinecone and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: httpx, langchain-core, langchain-openai, numpy, pinecone, pydantic, simsimd
Required-by: 


In [ ]:
!pip show pinecone-client

Name: pinecone-client
Version: 6.0.0
Summary: Pinecone client (DEPRECATED)
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: certifi, pinecone-plugin-interface, python-dateutil, typing-extensions, urllib3
Required-by: 


In [ ]:
from langchain_pinecone import Pinecone
from pinecone import Pinecone as PineconeClient
from pinecone import ServerlessSpec


index_name = "langchain-rag"
if index_name not in pinecone_client.list_indexes().names():
    pinecone_client.create_index(
        name=index_name,
        dimension=d,
        metric="cosine",
        spec=spec
    )
    print(f"Índice '{index_name}' criado no Pinecone.")

    pinecone_db = Pinecone.from_documents(
        documentos_empresa,
        embeddings,
        index_name=index_name
    )

    print(f"Documentos adicionados ao índice '{index_name}'")

else:
    print(f"Conectando ao índice existente '{index_name}'.")
    pinecone_db = Pinecone.from_existing_index(
        index_name,
        embeddings
    )

Conectando ao índice existente 'langchain-rag'.


In [ ]:
if pinecone_db:
  # Busca por similaridade
  pergunta_ti = "Como configuro a VPN?"
  resultados_pinecone = pinecone_db.similarity_search(pergunta_ti, k=2)
  print(f"\n Pergunta: {pergunta_ti}")
  print(f"\n Documentos mais relevantes (Pinecone):")
  for doc in resultados_pinecone:
    print(f"- {doc.page_content}")
    print(f"(Metadados: {doc.metadata})")

  # Busca com filtro (Pinecone também suporta)
  resultados_pinecone_filtrados = pinecone_db.similarity_search(
      "informações sobre regras",k=2,filter={"tipo": "politica"})

  print(f"\n Pergunta: 'Informações sobre regras' com filtro para tipo='politica'")
  print(f"\n Documentos relevantes e filtrados (Pinecone):")
  for doc in resultados_pinecone_filtrados:
    print(f"- {doc.page_content}")
    print(f"(Tipo: {doc.metadata['tipo']})")


 Pergunta: Como configuro a VPN?

 Documentos mais relevantes (Pinecone):

 Pergunta: 'Informações sobre regras' com filtro para tipo='politica'

 Documentos relevantes e filtrados (Pinecone):
